In [1]:
from dotenv import dotenv_values
import os

config = dotenv_values(".env.secret")  # load sensitive variables
print(config.keys())

for c, v in config.items():
    os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME', 'WANDB_NOTES', 'WANDB_NOTEBOOK_NAME'])


In [2]:
import wandb

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
import torch

torch.cuda.device_count()

1

In [5]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [6]:
import os

max(1, os.cpu_count() -1)

11

In [7]:
import os
import wandb
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from src.model import NeuralNetwork
from src.data import HousePricingDataModule

project_name = os.environ["WANDB_NAME"]

patience = 50
max_epochs = 500
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

# Setting up the training configuration
config = {
    "accelerator": accelerator,
    "max_epochs": max_epochs,
    "patience": patience
}

for _, batch_size in enumerate([256, 512]):

    for i in range(3):

        run_name = f"batch_size_{batch_size}_iter_{i + 1}"

        # Preparing data to be used
        data_module = HousePricingDataModule(batch_size=batch_size)
        data_module.prepare_data()

        # Requesting to data module the number of different features that are in the data
        in_features = data_module.in_features()

        config["batch_size"] = batch_size
        config["in_features"] = in_features

        wandb.init(
            job_type="training",
            name=run_name,
            project=project_name,
            config=config,
        )

        # Defining the model to be training
        model = NeuralNetwork(input_size=wandb.config["in_features"])

        # Defining the logger instance the lighning will use as default logging
        logger = WandbLogger(
            project=project_name,
            log_model="all",
            checkpoint_name=f"housepricing-{wandb.run.id}"
        )

        # Define how the model registry work
        checkpoint_callback = ModelCheckpoint(
            every_n_epochs=1,
            monitor="val_loss",
            mode = "min"
        )

        # Defining early stop configuration
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            mode="min",
            verbose=True,
            patience=wandb.config["patience"]
        )

        # Defines the training instance
        trainer = Trainer(
            accelerator=wandb.config["accelerator"],
            max_epochs=wandb.config["max_epochs"],
            logger=logger,
            log_every_n_steps=1,
            callbacks=[checkpoint_callback, early_stopping_callback]
        )

        trainer.fit(model, datamodule=data_module)

        wandb.finish()

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)
Problem at: /tmp/ipykernel_56465/403202778.py 38 <module>


KeyboardInterrupt: 

In [ ]:

        trainer.test(model, datamodule=data_module)
        preds = trainer.predict(model, datamodule=data_module)

        print(preds[0][:5])

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 1
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 239.16it/s]

UsageError: Run (breyz3rs) is finished. The call to `log` will be ignored. Please make sure that you are using an active run.